In [35]:
import django, os, sys, time, resource, re, gc, shutil
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from django.db.models import Count, Avg
from multiprocess import Pool
from functools import partial
import numpy as np
from functools import partial
from scipy.sparse import coo_matrix, csr_matrix, find, tril
import networkx as nx
import csv

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
qid = 1457

In [2]:
runs = [282, 283, 284, 285, 287]


fname = "../../data/topic_data/run_{}.xlsx".format(runs[0])

writer = pd.ExcelWriter(fname)

for i in range(1,len(runs)):

    topics1 = DynamicTopic.objects.filter(run_id=runs[i-1])
    topics2 = DynamicTopic.objects.filter(run_id=runs[i])

    df = pd.DataFrame.from_dict(list(topics2.values('title','id')))
    df['similarity'] = 0
    df['stopic'] = ""

    for t in topics2:
        scores = [0]
        titles = [""]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)

        
        m = max(scores)
        df.loc[df['id']==t.id,'similarity'] = m
        df.loc[df['id']==t.id,'stopic'] = titles[scores.index(max(scores))]


    df = df.sort_values('similarity',ascending=False)
    df.head()
    
    
    

    
    df.to_excel(writer,"run_{}".format(runs[i]))
    
writer.save()

In [3]:
runs = list(range(295,300))

writer = pd.ExcelWriter(fname)
fname = "../../data/topic_data/run_{}.xlsx".format(runs[0])

for i in range(1,len(runs)):

    topics1 = Topic.objects.filter(run_id=runs[i-1])
    topics2 = Topic.objects.filter(run_id=runs[i])

    df = pd.DataFrame.from_dict(list(topics2.values('title','id')))
    df['similarity'] = 0
    df['stopic'] = ""

    for t in topics2:
        scores = [0]
        titles = [""]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)

        
        m = max(scores)
        df.loc[df['id']==t.id,'similarity'] = m
        df.loc[df['id']==t.id,'stopic'] = titles[scores.index(max(scores))]


    df = df.sort_values('similarity',ascending=False)
    df.head()
    
    
    
    
    

    
    df.to_excel(writer,"run_{}".format(runs[i]))
    
writer.save()

In [36]:
runs = list(range(301,308))

#runs = [295,296,297]

col1s = []
col2s = []
ss = []
scols = []

for i in range(1,len(runs)):

    topics1 = Topic.objects.filter(run_id=runs[i-1])
    topics2 = Topic.objects.filter(run_id=runs[i])
    

    df = pd.DataFrame.from_dict(list(topics2.values('title','score')))
    
    df2 = pd.DataFrame.from_dict([{'title': 'None','score': 0}])
    
    df = df.append(df2)

    
    
    col1 = "topics_{}".format(runs[i-1])
    
    scol = "scores_{}".format(runs[i])
    bscol = "scores_{}".format(runs[i-1])
    
    if i==1:
        scols.append(bscol)
    
    col1s.append(col1)
    scols.append(scol)
    
    col2 = "topics_{}".format(runs[i])
    
    col2s.append(col2) 
    
    s = "similarity_{}-{}".format(runs[i-1],runs[i])
    ss.append(s)
    
    df = df.rename(columns = {'title': col2, 'score': scol})

    df[s] = 0
    df[col1] = "None"
    df[bscol] = 0

    for t in topics2:
        scores = [0]
        titles = [""]
        tscores = [0]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)
                tscores.append(ct.score)

        
        m = max(scores)
        df.loc[df[col2]==t.title, s] = m
        if m==0:
            df.loc[df[col2]==t.title, col1] = 'None'
        else:
            df.loc[df[col2]==t.title, col1] = titles[scores.index(max(scores))]
            df.loc[df[col2]==t.title, bscol] = tscores[scores.index(max(scores))]
            

    if i==1:   
        #df = pd.DataFrame.from_dict(list(topics2.values('title')))
        mdf = df
    else:
        mdf = mdf.merge(df,how="outer").fillna("")
    

columns = []
for i in range(len(col1s)):
    columns.append(col1s[i])
    columns.append(scols[i])
    columns.append(ss[i])
    if i == len(col1s)-1:
        columns.append(col2s[i])
        columns.append(scols[i+1])

print(columns)

mdf = mdf[columns]

mdf.head()



['topics_301', 'scores_301', 'similarity_301-302', 'topics_302', 'scores_302', 'similarity_302-303', 'topics_303', 'scores_303', 'similarity_303-304', 'topics_304', 'scores_304', 'similarity_304-305', 'topics_305', 'scores_305', 'similarity_305-306', 'topics_306', 'scores_306', 'similarity_306-307', 'topics_307', 'scores_307']


,topics_301,scores_301,similarity_301-302,topics_302,scores_302,similarity_302-303,topics_303,scores_303,similarity_303-304,topics_304,scores_304,similarity_304-305,topics_305,scores_305,similarity_305-306,topics_306,scores_306,similarity_306-307,topics_307,scores_307
0,"{plant, growth, root}",62.447042,6,"{plant, growth, root}",42.150649,9,"{plant, root, stress}",29.3845,8,"{plant, root, stress}",54.5742,8,"{plant, root, stress}",19.7081,7,"{plant, root, medicin}",18.1901,10,"{plant, root, medicin}",5.90624
1,"{plant, growth, root}",62.447042,6,"{plant, growth, root}",42.150649,9,"{plant, root, stress}",29.3845,8,"{plant, root, stress}",54.5742,8,"{plant, root, stress}",19.7081,3,"{gene, resist, diseas}",15.7966,9,"{gene, resist, diseas}",66.8996
2,"{china, growth, econom}",89.103673,10,"{china, growth, industri}",64.565867,3,"{develop, countri, sustain}",29.4608,8,"{develop, countri, resourc}",22.8286,4,"{countri, global, nation}",106.237,,,,,,
3,"{crop, yield, fertil}",77.810607,10,"{crop, yield, fertil}",33.993578,3,"{rice, yield, grain}",44.4743,,,,,,,,,,,,
4,"{climat, chang, adapt}",61.702157,3,"{risk, flood, vulner}",23.489525,,,,,,,,,,,,,,,


In [37]:
#col1s.append(col2)


#mdf = mdf.sort_values(columns,ascending=False).reset_index()


res = mdf.groupby(columns)



res = res.apply(lambda x: x.sort_values(s,ascending=False))

l = len(res)




fname = "../../data/topic_data/run_compare_{}_{}.xlsx".format(runs[0],runs[len(runs)-1])

writer = pd.ExcelWriter(fname, engine='xlsxwriter')

res.to_excel(writer)

worksheet = writer.sheets['Sheet1']

for i in range(len(ss)):
    c = chr(ord('A')-1+(i+1)*3)
    r = "{}2:{}{}".format(c,c,len(res))
    
    worksheet.conditional_format(r, {'type': '3_color_scale'})

writer.save()

res.head() 
    
    
res.head()

topics_301  \
topics_301              scores_301 similarity_301-302 topics_302                  scores_302 similarity_302-303 topics_303                  scores_303 similarity_303-304 topics_304                   scores_304 similarity_304-305 topics_305                   scores_305       similarity_305-306 topics_306                   scores_306       similarity_306-307 topics_307                   scores_307                                      
None                    0.000000   0                  None                        0.000000   0.0                None                        0.000000   0.0                None                         0.000000   0.0                None                         0.0              0.0                None                         0.0              0.0                None                         0.0              151                     None   
                                                                                                                                                                                                                                                                                                                                                                       }                            0.0              150                     None   
{acid, lignin, extract} 87.338937  6                  {lignin, cellulos, extract} 33.856348  9.0                {lignin, extract, pretreat} 65.768103  4.0                {extract, solvent, liquid}   15.414533                                                                                                                                                                                                     11   {acid, lignin, extract}   
                                                                                                                                                       6.0                {lignin, cellulos, pretreat} 32.737442  7.0                {lignin, pretreat, cellulos} 44.0606523872146 3.0                {extract, solvent, liquid}   5.40803021938341 10.0               {extract, solvent, liquid}   6.73165383975586 135  {acid, lignin, extract}   
                                                                                                                                                                                                                                                                                   7.0                {lignin, pretreat, cellulos} 22.3254762240993 10.0               {lignin, pretreat, cellulos} 16.0265737798418 136  {acid, lignin, extract}   

                                                                                                                                                                                                                                                                                                                                                                                                                          scores_301  \
topics_301              scores_301 similarity_301-302 topics_302                  scores_302 similarity_302-303 topics_303                  scores_303 similarity_303-304 topics_304                   scores_304 similarity_304-305 topics_305                   scores_305       similarity_305-306 topics_306                   scores_306       similarity_306-307 topics_307                   scores_307                         
None                    0.000000   0                  None                        0.000000   0.0                None                        0.000000   0.0                None                         0.000000   0.0                None                         0.0              0.0                None                         0.0              0.0                None                         0.0              151    0.000000   
                                                                                           

In [ ]:
print(mdf)

In [ ]:
runs = list(range(295,300))

runs = []

for i in range(len(runs)):

    topics = Topic.objects.filter(run_id=runs[i])
    
    tree = []
    
    for t in topics:
        
        tdict = {
            "parent_similarity": 0,
            "run": runs[i],
            "ntopics": topics.count(),
            "title": t.title,
            "sim_topics": [],
            "child_topics": 0
        }
        
        if i==0:
            tree.append(tdict)
        else:
            
            scores = [0]
            titles = [""]
            for ct in comparison_topics:
                score = len(set(t.top_words).intersection(set(ct.top_words)))
                if score>0:
                    scores.append(score)
                    titles.append(ct.title)


                parent = titles[scores.index(max(scores))]
                
                
                
                print(m)

In [ ]:
def find_similar(x):
    return(1,2)

df['bla'], df['x'] = zip(df.map(find_similar))
                         
df.head()